使用requests库、pyquery库下载、抽取 可可英语双语新闻 科技分类下的文章列表第1页和其中的所有文章信息（包括元数据）； 
将上述抽取的文章数据，以json字符串格式存入文件中。
要求存入文件的json字符串是由每篇文章作为一个字典构成的列表转化而来。

In [4]:
import requests
import random
import time
from pyquery import PyQuery as pq

class ProxyPool():
    def __init__(self):
        # 初始化读取proxy站点配置文件

        # 初始化读取proxy池存储位置（文件、数据库、NoSQL...)

        # 定时扫描proxy可用性、删除失效代理
        pass

    def check_a_proxy(self):
        pass

class KKBaseDownloader():
    def __init__(self):
        # 初始化代理池对象
        self.proxyp = ProxyPool()
        # 初始化headers配置列表文件路径
        self.headers_cfg_pth=''
        # 初始化最小、最大暂停间隔
        self.interval_min_max = (5,30)
        pass
    
    def gen_an_ua(self):
        self.ua_list = ["Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
        "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
        "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
        "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
        "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
        "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0"]

        return random.choice(self.ua_list) #与get_a_proxy作用相同

    def get_a_proxy(self):
        proxys = [

        ]
        idx = random.randint(1,len(proxys))
        return proxys[idx]

class KKNavDl(KKBaseDownloader):
    def __init__(self,init_url): #初始化
        super(KKNavDl,self).__init__()
        self.url_tgt = init_url

    def fetch_html(self):
        ua = self.gen_an_ua()
        headers = {'User-Agent':ua}
        # _proxy = self.get_a_proxy()
        # r = requests.get(self.url_tgt,proxies=_proxy)
        r = requests.get(self.url_tgt,headers=headers)
        if r.status_code==200:
            if r.encoding == 'ISO-8859-1':
                encodings = requests.utils.get_encodings_from_content(r.text)
                if encodings:
                    encoding = encodings[0]
                else:
                    encoding = r.apparent_encoding
            encode_content = r.content.decode(encoding, 'replace').encode('utf-8', 'replace')
            return encode_content
        else:
            return ''

class KKBaseExtractor():
    def __init__(self,html='<html></html>'):
        self.doc = pq(html)

class KKNavExt(KKBaseExtractor):
    def __init__(self,html):
        super(KKNavExt,self).__init__(html)

    def parse(self):
        lis = self.doc('#menu-list>li')
        nav_links = []
        for i in range(len(lis)):
            pq_li = pq(lis[i])
            title = str(pq_li.find('h2 a').attr('title'))
            t=len(title)
            l=len(str(pq_li('li').text()))
            p=len(str(pq_li('p').text()))
            abstract=str(pq_li('li').text()[t+1:l-p-1])
            date = str(pq_li.find('p').text()[0:10])
            author = str(pq_li.find('p').text()[14:19])
            sort = str(pq_li.find('span a').text())
            link = str(pq_li.find('h2 a').attr('href'))
            NewsInfo={'标题':title, '摘要':abstract, '日期':date, '编辑':author, '标签':sort, '链接':link}
            nav_links.append(NewsInfo)
        return nav_links

In [5]:
navDl = KKNavDl(init_url='http://www.kekenet.com/read/news/keji')
html = navDl.fetch_html()
navExt = KKNavExt(html)
nav_links = navExt.parse()
for lnk in nav_links:
    print(lnk)

{'标题': '小米推出智能手表Mi Watch', '摘要': '小米已在国内市场推出第一款智能手表Mi Watch，外观和Apple Watch特别像。', '日期': '2019-11-18', '编辑': 'Kelly', '标签': '智能手表 小米 双语新闻', '链接': 'http://www.kekenet.com/read/201911/599584.shtml'}
{'标题': '新型神经网络可以用快1亿倍速度解决"三体问题"', '摘要': '三体问题是物理学中最复杂的计算题之一，但它在人工智能领域可能遇到了对手：一种新型神经网络有望以比现有技术快1亿倍的速度找出其解决方案。', '日期': '2019-11-14', '编辑': 'Kelly', '标签': '双语新闻 神经网络 三体问题', '链接': 'http://www.kekenet.com/read/201911/599412.shtml'}
{'标题': '14岁女孩发明可替代抗生素的液体绷带', '摘要': '一个14岁的科学家发明了一种可替代抗生素的液体绷带，在全国科学竞赛上赢得了25,000美元奖金。', '日期': '2019-11-13', '编辑': 'Kelly', '标签': '双语新闻 液体绷带 抗生素', '链接': 'http://www.kekenet.com/read/201911/599335.shtml'}
{'标题': '特斯拉推出新版太阳能屋顶', '摘要': '特斯拉推出了第三代家用太阳能光伏瓦。', '日期': '2019-11-07', '编辑': 'Kelly', '标签': '双语新闻 特斯拉 太阳能屋顶', '链接': 'http://www.kekenet.com/read/201911/598810.shtml'}
{'标题': '苹果推出AirPods Pro无线降噪耳机', '摘要': '苹果推出AirPods Pro无线降噪耳机，形似豌豆射手。', '日期': '2019-11-05', '编辑': 'Kelly', '标签': '双语新闻 苹果 无线降噪耳机', '链接': 'http://www.kekenet.com/read/201911/598555.shtml'}
{'标题': '新的基因编辑技术诞生', '

In [6]:
import json
json_f = open('json_output.txt','w+')
json.dump(nav_links,json_f)
json_f.close()